In [4]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from scipy.io import savemat
import math
from loading_real_wave_noise import loading_real_wave_noise
from Reading_path_test import loading_paths_from_MAT
from Control_filter_selection import Control_filter_selection
from FxNLMS_algorithm import FxNLMS, train_fxnlms_algorithm
from Disturbance_generation import Disturbance_generation_from_real_noise
from Combine_SFANC_with_FxNLMS import SFANC_FxNLMS

print(torch.cuda.is_available())

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# real noises
fs = 16000
StepSize = 0.0001
sound_name = 'Mix_Aircraft_Traffic'
waveform, resample_rate = loading_real_wave_noise(folde_name='Real Noise Examples/', sound_name=sound_name+'.wav')

C:\Users\devan\AppData\Roaming\Python\Python313\site-packages\torchaudio\_backend\utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(


In [ ]:
Pri_path, Secon_path = loading_paths_from_MAT(folder='Pz and Sz', subfolder='Dongyuan', Pri_path_file_name='Primary_path.mat', Sec_path_file_name='Secondary_path.mat')
Dis, Fx, Re = Disturbance_generation_from_real_noise(fs=fs, Repet=0, wave_form=waveform, Pri_path=Pri_path, Sec_path=Secon_path)
# Dis: disturbance (cotrolled noise)， Fx: fixed-x signal, Re: repeated waveform (primary_noise) Repetition=Repet+1

NameError: name 'loading_paths_from_MAT' is not defined

In [ ]:
print(waveform.shape)
print(Re.shape)
print(Dis.shape)

import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 10000

In [ ]:
# FxNLMS

controller = FxNLMS(Len=1024) # 1024 is the same size of coeffient vector of fixed-filter
ErrorFxNLMS = train_fxnlms_algorithm(Model=controller, Ref=Fx, Disturbance=Dis, Stepsize=StepSize)

plt.title('The FxNLMS Algorithm')
plt.plot(Time, Dis, color='blue', label='ANC off')
plt.plot(Time, ErrorFxNLMS, color='green', label='ANC on')
plt.ylabel('Magnitude')
plt.xlabel('Time (seconds)')
plt.legend()
plt.grid()
plt.savefig('FxNLMS.pdf', dpi=600, bbox_inches='tight', pad_inches=0)
plt.show()

In [ ]:
# Combine SFANC with FxNLMS

# prediction index
id_vector = Control_filter_selection(fs=16000, Primary_noise=Re.unsqueeze(0)) # Primary_noise: torch.Size([1, XX])
print(id_vector)

# Using prediction index in SFANC_FxNLMS
FILE_NAME_PATH = 'Trained models/Pretrained_Control_filters.mat'
SFANC_FxNLMS_Cancellation = SFANC_FxNLMS(MAT_FILE=FILE_NAME_PATH, fs=16000)
Error_SFANC_FxNLMS = SFANC_FxNLMS_Cancellation.noise_cancellation(Dis=Dis, Fx=Fx, filter_index=id_vector, Stepsize=StepSize)

plt.title('The Hybrid SFANC-FxNLMS Algorithm')
plt.plot(Time, Dis, color='blue', label='ANC off')
plt.plot(Time, Error_SFANC_FxNLMS, color='red', label='ANC on')
plt.ylabel('Magnitude')
plt.xlabel('Time (seconds)')
plt.legend()
plt.grid()
plt.savefig('SFANC_FxNLMS.pdf', dpi=600, bbox_inches='tight', pad_inches=0)
plt.show()